In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### function of data normalization
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing

def min_max_normalize(my_matrix):
    scaler = MinMaxScaler()
    scaler.fit(my_matrix)
    my_matrix_normorlize=scaler.transform(my_matrix)
    return my_matrix_normorlize

def normalize(my_matrix):
    scaler = preprocessing.StandardScaler().fit(my_matrix)
    my_matrix_normorlize=scaler.transform(my_matrix)
    return my_matrix_normorlize

In [2]:
def get_onehot_list(train_list):
    tr_one = {}
#     tr_two = {}
#     tr_three = {}
    for item in train_list:
        tr_one[item] = 1
    i = 0
    for item in tr_one.keys():
        tmp = np.zeros(len(tr_one))
        tmp[i] = 1
        tr_one[item] = tmp
        i += 1
    return tr_one

def text_to_onehot(item):
#     tmp = item.split('-')
    onehot_data = text_onehot_list[item]
    return onehot_data

In [3]:
columns = ['Rotation_speed','Total_rate','one','two','three','RTD','Temperature']
train =pd.read_csv('C:\\.ipynb_checkpoints\\training_data700.csv', names=columns)
print(len(np.array(train)))
train.head()
train.Temperature=pd.to_numeric(train.Temperature)

3200


In [4]:
### normalize speed
speed = np.array([train.Rotation_speed.values],dtype = np.float32).transpose(1,0)
print(speed.shape)
speed_min = min_max_normalize(speed)
print(speed_min[:4])
speed_nor = normalize(speed)
# print(speed_nor[:4])

### normalize total rate
Total_rate = np.array([train.Total_rate.values],dtype = np.float32).transpose(1,0)
print(Total_rate.shape)
Total_rate_min = min_max_normalize(Total_rate)
print(Total_rate_min[:4])
Total_rate_nor = normalize(Total_rate)
# print(speed_nor[:4])

(3200, 1)
[[0.03999996]
 [0.3399999 ]
 [0.71000004]
 [0.88999987]]
(3200, 1)
[[0.13333333]
 [0.16666663]
 [0.66666675]
 [0.73333335]]


In [5]:
### get the text_onehot_list for transfer string to one-hot vector
train_one = np.concatenate((train.one.values,train.two.values,train.three.values))
# test_one = np.concatenate((test.one.values,test.two.values,test.three.values))
text_onehot_list = get_onehot_list(train_one)
print(text_onehot_list[train_one[20]])
print(text_to_onehot(train_one[20]).shape)

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
(30,)


In [6]:
### prepare the training data
quantity = np.concatenate((speed_min,Total_rate_min),1)
#print(quantity.shape)
# (3140,2)
train_y = np.concatenate((np.array([train.RTD.values]).transpose(1,0),
                         np.array([train.Temperature.values]).transpose(1,0)),axis = 1)

#print(train_y.shape)
# (3140,2)
train_one = np.array(train.one.values)
train_two = np.array(train.two.values)
train_three = np.array(train.three.values)

test_yR = train_y[2800:3201,0:1]
test_yT = train_y[2800:3201,1:2]
#print(len(test_y),test_y.shape)
# (340,2)

test_quantity = quantity[2800:3201,:]
#print(test_quantity.shape)
# (340,2)

test_one = train_one[2800:3201]
test_two = train_two[2800:3201]
test_three = train_three[2800:3201]
#print(test_one.shape)
# (340)

train_yR = train_y[0:2800,0:1]
train_yT = train_y[0:2800,1:2]
#print(len(train_y))
#2800
quantity = quantity[0:2800,:]

train_one = train_one[0:2800]
train_two = train_two[0:2800]
train_three = train_three[0:2800]


In [ ]:
training_data = np.concatenate((np.array([train.RTD.values]).transpose(1,0),
                         np.array([train.Temperature.values]).transpose(1,0),
                        np.array([train.one.values]).transpose(1,0),np.array([train.two.values]).transpose(1,0),
                        np.array([train.three.values]).transpose(1,0),speed_min,Total_rate_min),axis = 1)
print(training_data.shape)
print(type(train_y))

In [7]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision


# hyperparameters
EPOCH = 70
BATCH_SIZE = 16
LR = 0.002

class Extrader(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Extrader, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.output_size = output_size
        # 压缩
        self.model = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.Tanh(),
#             nn.Linear(64, 16),
            nn.Tanh(),
#             nn.Linear(64, 16),
#             nn.Tanh(),
            nn.Linear(hidden_size, output_size),   # 压缩成3个特征, 进行 3D 图像可视化
        )
        
    def forward(self, x):
        return(self.model(x))

In [8]:
class AutoEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(AutoEncoder, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.output_size = output_size
        # 压缩
        self.encoder = nn.Sequential(
#             nn.Linear(5, 32),
            nn.Linear(input_size, hidden_size),
            nn.Tanh(),
#             nn.Linear(15, 15),
#             nn.Tanh(),
#             nn.Linear(15, 15),
#             nn.Tanh(),
            nn.Linear(hidden_size, 1),   # 压缩成3个特征, 进行 3D 图像可视化
#             nn.Tanh(),
        )
        # 解压
        self.decoder = nn.Sequential(
            nn.Linear(1, hidden_size),
            nn.Tanh(),
#             nn.Linear(15, 15),
#             nn.Tanh(),
#             nn.Linear(15, 15),
#             nn.Tanh(),
            nn.Linear(hidden_size,output_size),
#             nn.Sigmoid(),       # 激励函数让输出值在 (0, 1)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [9]:
class IntegratedModel(nn.Module):
    def __init__(self):
        super(IntegratedModel, self).__init__()
        self.AE = AutoEncoder(30,15,30)
        self.extrater = Extrader(5,64,1)
        
    def forward(self, quality, quantity):
        encoded_1,decoded_1 = self.AE(quality[0].unsqueeze(0))
#         print(encoded_1.shape)
        encoded_2,decoded_2 = self.AE(quality[1].unsqueeze(0))
        encoded_3,decoded_3 = self.AE(quality[2].unsqueeze(0))
#         print(quantity[0,0].shape)
        input_value = torch.cat((encoded_1, encoded_2,
                                 encoded_3,quantity),1)
#         print(input_value.shape)
        out = self.extrater(input_value)
        return decoded_1,decoded_2,decoded_3,out

In [10]:
### training the integrated model
my_model = IntegratedModel()
EPOCH = 100
BATCH_SIZE = 16
LR = 0.002
optimizer = torch.optim.Adam(my_model.parameters(), lr=LR)
# EX_optimizer = torch.optim.Adam(my_model.extrater.parameters(), lr=LR)
loss_func = nn.MSELoss()

iteration = 0
AE_loss = []
MSE_loss = []
embeddingvalue = []

#RTD
for epoch in range(EPOCH):
#     total_loss1 = 0
#     total_loss2 = 0
    for i in range(len(train_yR)):
        ### train AE
        iteration += 1
        targetR = torch.FloatTensor(train_yR[i]).view(-1,1).detach()
        en_input1 = torch.FloatTensor(text_to_onehot(train_one[i])).unsqueeze(0)
        en_input2 = torch.FloatTensor(text_to_onehot(train_two[i])).unsqueeze(0)
        en_input3 = torch.FloatTensor(text_to_onehot(train_three[i])).unsqueeze(0)
        x = torch.cat((en_input1,en_input2,en_input3),0)
        y = torch.FloatTensor(quantity[i]).unsqueeze(0)
        d1,d2,d3,outR = my_model(x,y)
        
        loss = loss_func(d1, en_input1) + loss_func(d2, en_input2) + loss_func(d3, en_input3)
        optimizer.zero_grad()               # clear gradients for this training step
        loss.backward()                     # backpropagation, compute gradients
        optimizer.step()                    # apply gradients
#         total_loss1 += loss.item()
#         loss1 = total_loss1/2800
        if (iteration % 50 == 0):
            AE_loss.append(loss.item())

        ### train extrater
        targetR = torch.FloatTensor(train_yR[i]).view(-1,1).detach()
        en_input1 = torch.FloatTensor(text_to_onehot(train_one[i])).unsqueeze(0)
        en_input2 = torch.FloatTensor(text_to_onehot(train_two[i])).unsqueeze(0)
        en_input3 = torch.FloatTensor(text_to_onehot(train_three[i])).unsqueeze(0)
        x = torch.cat((en_input1,en_input2,en_input3),0)
        y = torch.FloatTensor(quantity[i]).unsqueeze(0)
        d1,d2,d3,outR = my_model(x,y)
        
        loss = loss_func(outR, targetR)
        optimizer.zero_grad()               # clear gradients for this training step
        loss.backward()                     # backpropagation, compute gradients
        optimizer.step()                    # apply gradients
#         total_loss2 += loss.item()
#         loss2 = total_loss2/2800
#         print('epoch',epoch,loss.item())
        if (iteration % 50 == 0):
            MSE_loss.append(loss.item())

        if(iteration % 50 == 0):
            for item in text_onehot_list.keys():
                input_ = torch.FloatTensor(text_onehot_list[item]).unsqueeze(0)
                e1,d1 = my_model.AE(input_)
                embeddingvalue.append(e1)
                
    if(epoch%10 == 0):
        print(loss.item())

0.08722434192895889
0.01923835277557373
0.027203591540455818
0.001781813451088965
0.023908281698822975
0.010273528285324574
0.0025249719619750977
0.025413524359464645
0.00687146931886673
0.02978048287332058


In [12]:
print(AE_loss[0:56])
print(len(AE_loss))

[0.10946527123451233, 0.09820139408111572, 0.1089572086930275, 0.10487370938062668, 0.08161215484142303, 0.08684028685092926, 0.10713519901037216, 0.09555432200431824, 0.09189888834953308, 0.09336911886930466, 0.08387342095375061, 0.09453633427619934, 0.09722346067428589, 0.09673546254634857, 0.1116456538438797, 0.09139372408390045, 0.08639788627624512, 0.08086403459310532, 0.09939008206129074, 0.09186556190252304, 0.09945819526910782, 0.10670575499534607, 0.08172393590211868, 0.08934473991394043, 0.10000390559434891, 0.08989650011062622, 0.10071516782045364, 0.08004037290811539, 0.10258973389863968, 0.10217271745204926, 0.09167755395174026, 0.10524097084999084, 0.0844777375459671, 0.08343097567558289, 0.1059490293264389, 0.08148669451475143, 0.09224654734134674, 0.0934741273522377, 0.10561081022024155, 0.03897746279835701, 0.10724280774593353, 0.0937584862112999, 0.09951390326023102, 0.0833386778831482, 0.1001085415482521, 0.08915038406848907, 0.10619117319583893, 0.09664888679981232,

In [13]:
print(MSE_loss[0:56])
print(len(MSE_loss))

[74.34282684326172, 5.755787372589111, 2.8454840183258057, 7.423714637756348, 2.6082303524017334, 0.6276172399520874, 7.072073459625244, 0.9920507669448853, 0.8900492787361145, 22.101322174072266, 11.758752822875977, 0.05038941279053688, 4.151065826416016, 3.9812536239624023, 8.048303604125977, 3.5507166385650635, 0.14415304362773895, 9.042536735534668, 2.131591796875, 1.5128308534622192, 5.502877712249756, 3.788783550262451, 0.4275380074977875, 0.646908164024353, 2.463491916656494, 5.9607720375061035, 1.162295937538147, 13.021227836608887, 1.6737836599349976, 5.401709079742432, 0.5333505272865295, 1.784830927848816, 0.45003560185432434, 3.4301698207855225, 1.10878586769104, 0.01789747178554535, 0.953041136264801, 0.4390407204627991, 2.5482289791107178, 0.5151699781417847, 1.4912757873535156, 0.17402249574661255, 0.12424344569444656, 0.21426597237586975, 0.7280038595199585, 1.1251683235168457, 3.361624002456665, 0.39096003770828247, 0.002744622528553009, 0.4378787577152252, 1.499275684

In [25]:
print(torch.FloatTensor(embeddingvalue[0:30]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[30:60]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[60:90]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[90:120]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[120:150]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[150:180]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[180:210]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[210:240]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[240:270]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[270:300]).view(-1,1))
## 10
print(torch.FloatTensor(embeddingvalue[300:330]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[330:360]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[360:390]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[390:420]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[420:450]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[450:480]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[480:510]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[510:540]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[540:570]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[570:600]).view(-1,1))
## 20
print(torch.FloatTensor(embeddingvalue[600:630]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[630:660]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[660:690]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[690:720]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[720:750]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[750:780]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[780:810]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[810:840]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[840:870]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[870:900]).view(-1,1))
## 30
print(torch.FloatTensor(embeddingvalue[900:930]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[930:960]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[960:990]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[990:1020]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1020:1050]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1050:1080]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1080:1110]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1110:1140]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1140:1170]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1170:1200]).view(-1,1))
## 40
print(torch.FloatTensor(embeddingvalue[1200:1230]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1230:1260]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1260:1290]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1290:1320]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1320:1350]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1350:1380]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1380:1410]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1410:1440]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1440:1470]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1470:1500]).view(-1,1))
## 50
print(torch.FloatTensor(embeddingvalue[1500:1530]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1530:1560]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1560:1590]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1590:1620]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1620:1650]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1650:1680]).view(-1,1))
## 56


## 2D and 3D embeddingvalue
# print(np.array(embeddingvalue[0:30]).reshape(30,1))
# print(np.array(embeddingvalue[30:60]).reshape(30,1))
# print(np.array(embeddingvalue[60:90]).reshape(30,1))
# print(np.array(embeddingvalue[90:120]).reshape(30,1))
# print(np.array(embeddingvalue[120:150]).reshape(30,1))
# print(np.array(embeddingvalue[150:180]).reshape(30,1))
# print(np.array(embeddingvalue[180:210]).reshape(30,1))
# print(np.array(embeddingvalue[210:240]).reshape(30,1))
# print(np.array(embeddingvalue[240:270]).reshape(30,1))
# print(np.array(embeddingvalue[270:300]).reshape(30,1))
# ## 10
# print(np.array(embeddingvalue[300:330]).reshape(30,1))
# print(np.array(embeddingvalue[330:360]).reshape(30,1))
# print(np.array(embeddingvalue[360:390]).reshape(30,1))
# print(np.array(embeddingvalue[390:420]).reshape(30,1))
# print(np.array(embeddingvalue[420:450]).reshape(30,1))
# print(np.array(embeddingvalue[450:480]).reshape(30,1))
# print(np.array(embeddingvalue[480:510]).reshape(30,1))
# print(np.array(embeddingvalue[510:540]).reshape(30,1))
# print(np.array(embeddingvalue[540:570]).reshape(30,1))
# print(np.array(embeddingvalue[570:600]).reshape(30,1))
# ## 20
# print(np.array(embeddingvalue[600:630]).reshape(30,1))
# print(np.array(embeddingvalue[630:660]).reshape(30,1))
# print(np.array(embeddingvalue[660:690]).reshape(30,1))
# print(np.array(embeddingvalue[690:720]).reshape(30,1))
# print(np.array(embeddingvalue[720:750]).reshape(30,1))
# print(np.array(embeddingvalue[750:780]).reshape(30,1))
# print(np.array(embeddingvalue[780:810]).reshape(30,1))
# print(np.array(embeddingvalue[810:840]).reshape(30,1))
# print(np.array(embeddingvalue[840:870]).reshape(30,1))
# print(np.array(embeddingvalue[870:900]).reshape(30,1))
# ## 30
# print(np.array(embeddingvalue[900:930]).reshape(30,1))
# print(np.array(embeddingvalue[930:960]).reshape(30,1))
# print(np.array(embeddingvalue[960:990]).reshape(30,1))
# print(np.array(embeddingvalue[990:1020]).reshape(30,1))
# print(np.array(embeddingvalue[1020:1050]).reshape(30,1))
# print(np.array(embeddingvalue[1050:1080]).reshape(30,1))
# print(np.array(embeddingvalue[1080:1110]).reshape(30,1))
# print(np.array(embeddingvalue[1110:1140]).reshape(30,1))
# print(np.array(embeddingvalue[1140:1170]).reshape(30,1))
# print(np.array(embeddingvalue[1170:1200]).reshape(30,1))
# ## 40
# print(np.array(embeddingvalue[1200:1230]).reshape(30,1))
# print(np.array(embeddingvalue[1230:1260]).reshape(30,1))
# print(np.array(embeddingvalue[1260:1290]).reshape(30,1))
# print(np.array(embeddingvalue[1290:1320]).reshape(30,1))
# print(np.array(embeddingvalue[1320:1350]).reshape(30,1))
# print(np.array(embeddingvalue[1350:1380]).reshape(30,1))
# print(np.array(embeddingvalue[1380:1410]).reshape(30,1))
# print(np.array(embeddingvalue[1410:1440]).reshape(30,1))
# print(np.array(embeddingvalue[1440:1470]).reshape(30,1))
# print(np.array(embeddingvalue[1470:1500]).reshape(30,1))
# ## 50
# print(np.array(embeddingvalue[1500:1530]).reshape(30,1))
# print(np.array(embeddingvalue[1530:1560]).reshape(30,1))
# print(np.array(embeddingvalue[1560:1590]).reshape(30,1))
# print(np.array(embeddingvalue[1590:1620]).reshape(30,1))
# print(np.array(embeddingvalue[1620:1650]).reshape(30,1))
# print(np.array(embeddingvalue[1650:1680]).reshape(30,1))
# ## 56

tensor([[-1.2589],
        [-1.0897],
        [-1.1411],
        [-1.1095],
        [-1.1406],
        [-0.9945],
        [-0.9865],
        [-0.8909],
        [-0.8868],
        [-0.8686],
        [-1.1770],
        [-0.9519],
        [-0.8484],
        [-0.8570],
        [-0.8312],
        [-1.1024],
        [-0.7697],
        [-0.8018],
        [-1.1098],
        [-1.0637],
        [-0.6877],
        [-1.1211],
        [-0.9402],
        [-0.9630],
        [-1.1494],
        [-0.9824],
        [-0.8169],
        [-1.0119],
        [-0.9414],
        [-1.0297]])
tensor([[-2.5625],
        [-2.5134],
        [-2.4137],
        [-2.3357],
        [-2.5335],
        [-2.4152],
        [-2.3685],
        [-2.4862],
        [-2.1874],
        [-2.2297],
        [-2.3918],
        [-2.1117],
        [-2.1855],
        [-2.0652],
        [-1.8722],
        [-2.2418],
        [-1.8445],
        [-2.0729],
        [-2.5439],
        [-2.1894],
        [-1.9269],
        [-2.4293],
        [-2

        [-3.1483]])
tensor([[-3.7445],
        [-3.5086],
        [-3.2517],
        [-2.8787],
        [-3.1819],
        [-2.8666],
        [-3.1796],
        [-3.2791],
        [-3.0592],
        [-2.9799],
        [-3.3296],
        [-3.0184],
        [-3.0412],
        [-2.9555],
        [-3.2416],
        [-2.9630],
        [-3.2866],
        [-3.0025],
        [-3.3607],
        [-3.0444],
        [-3.3909],
        [-3.2517],
        [-3.0773],
        [-3.4252],
        [-3.5120],
        [-3.3917],
        [-3.3068],
        [-3.1698],
        [-2.8760],
        [-3.1706]])
tensor([[-3.8737],
        [-3.6377],
        [-3.3278],
        [-2.9646],
        [-3.2570],
        [-2.9398],
        [-3.2948],
        [-3.3569],
        [-3.1339],
        [-3.0012],
        [-3.4068],
        [-3.0921],
        [-3.1109],
        [-3.0570],
        [-3.3207],
        [-3.0492],
        [-3.3355],
        [-3.0703],
        [-3.4386],
        [-3.1180],
        [-3.4690],
        [-

        [-2.4831]])
tensor([[-3.7220],
        [-2.4966],
        [-2.7578],
        [-0.2445],
        [-0.5885],
        [-1.2277],
        [-2.7447],
        [-3.1572],
        [-2.3549],
        [-0.9486],
        [-2.2824],
        [-0.6961],
        [-1.9956],
        [-2.3578],
        [-2.6180],
        [-1.7685],
        [-1.8096],
        [-2.0973],
        [-2.7413],
        [-2.3443],
        [-2.2698],
        [-2.0306],
        [-2.5057],
        [-2.2682],
        [-3.0508],
        [-2.7980],
        [-2.3362],
        [-1.7293],
        [-1.8618],
        [-2.4011]])
tensor([[-3.7464],
        [-2.4727],
        [-2.7400],
        [-0.1120],
        [-0.8888],
        [-0.8053],
        [-2.5960],
        [-2.9906],
        [-2.2236],
        [-1.1419],
        [-2.0295],
        [-0.6410],
        [-1.9444],
        [-2.2445],
        [-2.2880],
        [-1.5613],
        [-1.7396],
        [-2.1846],
        [-2.0474],
        [-2.4036],
        [-2.1545],
        [-

        [-2.0795]])
tensor([[-3.9891],
        [-2.7736],
        [-2.7787],
        [ 0.0743],
        [-0.4659],
        [-1.0509],
        [-0.7990],
        [-1.1073],
        [-1.3225],
        [-0.8637],
        [-0.4788],
        [-1.0921],
        [-1.5454],
        [-1.8036],
        [-2.0588],
        [-1.3866],
        [-1.2584],
        [-1.1830],
        [-1.1622],
        [-1.3118],
        [-2.4254],
        [-1.5854],
        [-1.4518],
        [-1.7987],
        [-1.8980],
        [-1.6155],
        [-1.6287],
        [-1.4207],
        [-1.6244],
        [-1.9617]])
tensor([[-4.2200],
        [-2.9582],
        [-2.8952],
        [ 0.0485],
        [-0.3760],
        [-1.0629],
        [-1.2167],
        [-1.4441],
        [-1.8854],
        [-1.0648],
        [-0.7023],
        [-1.0307],
        [-1.6136],
        [-1.7517],
        [-2.1304],
        [-1.4663],
        [-1.1512],
        [-1.5055],
        [-1.4402],
        [-1.3841],
        [-2.5160],
        [-

In [26]:
#RTD
### testing the integrated model
batch_loss_list = []
tempeture_loss_list = []
RTD_loss_list = []

loss_func = nn.MSELoss()
with torch.no_grad():
    batch_loss = 0
    tempeture_loss = 0
    RTD_loss = 0
    SSRr = 0
    SSRt = 0
    for i in range(len(test_yR)):
        
        targetR = torch.FloatTensor(test_yR[i]).view(-1,1).detach()
        en_input1 = torch.FloatTensor(text_to_onehot(test_one[i])).unsqueeze(0)
        en_input2 = torch.FloatTensor(text_to_onehot(test_two[i])).unsqueeze(0)
        en_input3 = torch.FloatTensor(text_to_onehot(test_three[i])).unsqueeze(0)
        x = torch.cat((en_input1,en_input2,en_input3),0)
        y = torch.FloatTensor(test_quantity[i]).unsqueeze(0)
        d1,d2,d3,outR = my_model(x,y)

        batch_loss += loss_func(outR, targetR).item()
        RTD_loss += loss_func(outR[0], targetR[0]).item()
#         tempeture_loss += loss_func(out[0,1], target[0,1]).item()

        a=torch.ones(1)*16.5239
        SSRr += loss_func(outR[0],a).item()
#         b=torch.ones(1)*248.679
#         SSRt += loss_func(out[0,1],b).item()

#         batch_loss_list.append(loss_func(out, target).item())
#         tempeture_loss_list.append(loss_func(out[0,1], target[0,1]).item())
#         RTD_loss_list.append(loss_func(out[0,0], target[0,0]).item())

print(SSRr/(RTD_loss+SSRr))
print(RTD_loss/len(test_yR))

0.9615072016444469
0.1802837336968696


In [ ]:
print(len(text_onehot_list))
# print(text_onehot_list.keys())
output = []
for item in text_onehot_list.keys():
#     print(item,':',text_onehot_list[item])
    input_ = torch.FloatTensor(text_onehot_list[item]).unsqueeze(0)
    e1,d1 = my_model.AE(input_)
    print(e1)
    embed = e1.detach().numpy()[0]
    output.append([item, embed])
print(output)

In [27]:
### training the integrated model
my_model = IntegratedModel()
EPOCH = 100
BATCH_SIZE = 16
LR = 0.002
optimizer = torch.optim.Adam(my_model.parameters(), lr=LR)
# EX_optimizer = torch.optim.Adam(my_model.extrater.parameters(), lr=LR)
loss_func = nn.MSELoss()

iteration = 0
AE_loss = []
MSE_loss = []
embeddingvalue = []

#outlet temperature
for epoch in range(EPOCH):
#     total_loss1 = 0
#     total_loss2 = 0
    for i in range(len(train_yT)):
        ### train AE
        iteration += 1
        targetT = torch.FloatTensor(train_yT[i]).view(-1,1).detach()
        en_input1 = torch.FloatTensor(text_to_onehot(train_one[i])).unsqueeze(0)
        en_input2 = torch.FloatTensor(text_to_onehot(train_two[i])).unsqueeze(0)
        en_input3 = torch.FloatTensor(text_to_onehot(train_three[i])).unsqueeze(0)
        x = torch.cat((en_input1,en_input2,en_input3),0)
        y = torch.FloatTensor(quantity[i]).unsqueeze(0)
        d1,d2,d3,outT = my_model(x,y)
        
        loss = loss_func(d1, en_input1) + loss_func(d2, en_input2) + loss_func(d3, en_input3)
        optimizer.zero_grad()               # clear gradients for this training step
        loss.backward()                     # backpropagation, compute gradients
        optimizer.step()                    # apply gradients
#         total_loss1 += loss.item()
#         loss1 = total_loss1/2800
#         print(loss.item())
        if (iteration % 50 == 0):
            AE_loss.append(loss.item())

        ### train extrater
        targetT = torch.FloatTensor(train_yT[i]).view(-1,1).detach()
        en_input1 = torch.FloatTensor(text_to_onehot(train_one[i])).unsqueeze(0)
        en_input2 = torch.FloatTensor(text_to_onehot(train_two[i])).unsqueeze(0)
        en_input3 = torch.FloatTensor(text_to_onehot(train_three[i])).unsqueeze(0)
        x = torch.cat((en_input1,en_input2,en_input3),0)
        y = torch.FloatTensor(quantity[i]).unsqueeze(0)
        d1,d2,d3,outT = my_model(x,y)
        
        loss = loss_func(outT, targetT)
        optimizer.zero_grad()               # clear gradients for this training step
        loss.backward()                     # backpropagation, compute gradients
        optimizer.step()                    # apply gradients
#         total_loss2 += loss.item()
#         loss2 = total_loss2/2800
#         print(loss.item())
        if (iteration % 50 == 0):
            MSE_loss.append(loss.item())

        if(iteration % 50 == 0):
            for item in text_onehot_list.keys():
                input_ = torch.FloatTensor(text_onehot_list[item]).unsqueeze(0)
                e1,d1 = my_model.AE(input_)
                embeddingvalue.append(e1)
                
    if(epoch%10 == 0):
        print(loss.item())

26.483631134033203
0.26999199390411377
0.04148322716355324
0.06178489327430725
0.0791788324713707
0.20911474525928497
0.0644838809967041
0.03254033625125885
0.032589901238679886
0.0452120378613472


In [28]:
print(AE_loss[0:56])
print(len(AE_loss))

[0.10451320558786392, 0.08690999448299408, 0.09960921108722687, 0.10660754144191742, 0.08477061986923218, 0.08689121901988983, 0.10985799133777618, 0.09352338314056396, 0.09613561630249023, 0.09488918632268906, 0.08266007900238037, 0.09263255447149277, 0.09406419098377228, 0.0946711003780365, 0.11661143600940704, 0.09550371766090393, 0.0851304829120636, 0.07269042730331421, 0.10804605484008789, 0.09057578444480896, 0.11184002459049225, 0.11625779420137405, 0.07808403670787811, 0.08801769465208054, 0.10285801440477371, 0.08561398833990097, 0.10435374081134796, 0.08398598432540894, 0.09659568965435028, 0.09881697595119476, 0.09556563198566437, 0.10559167712926865, 0.09096090495586395, 0.08467672765254974, 0.10087642073631287, 0.0825105607509613, 0.10181517899036407, 0.0965501219034195, 0.10986793041229248, 0.06044424697756767, 0.114036925137043, 0.08836619555950165, 0.10576295852661133, 0.0835963785648346, 0.10357527434825897, 0.086846724152565, 0.10025610029697418, 0.09806947410106659, 

In [29]:
print(MSE_loss[0:56])
print(len(MSE_loss))

[55755.26171875, 50112.69140625, 50165.28125, 45673.39453125, 42861.76953125, 42594.48828125, 35004.453125, 38877.1171875, 29085.642578125, 33398.296875, 27474.994140625, 24228.193359375, 23981.728515625, 21625.6875, 22012.40625, 19108.79296875, 18252.31640625, 17336.53515625, 15492.6005859375, 15417.1201171875, 12400.216796875, 12432.185546875, 10996.7529296875, 11012.5830078125, 6762.587890625, 7802.40234375, 5349.2119140625, 8112.74072265625, 5238.48291015625, 5092.7919921875, 3847.957275390625, 2984.369140625, 2974.6875, 2456.62890625, 2823.911376953125, 1828.3427734375, 1774.1890869140625, 1900.3697509765625, 1042.7421875, 1041.33056640625, 1172.4656982421875, 509.7917175292969, 906.2841186523438, 996.3657836914062, 311.4624938964844, 482.4349365234375, 105.6495132446289, 331.7877502441406, 289.4520568847656, 2.638493061065674, 199.78082275390625, 14.522477149963379, 10.535440444946289, 5.070468425750732, 0.7081596255302429, 26.483631134033203]
5600


In [30]:
print(torch.FloatTensor(embeddingvalue[0:30]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[30:60]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[60:90]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[90:120]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[120:150]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[150:180]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[180:210]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[210:240]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[240:270]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[270:300]).view(-1,1))
## 10
print(torch.FloatTensor(embeddingvalue[300:330]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[330:360]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[360:390]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[390:420]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[420:450]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[450:480]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[480:510]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[510:540]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[540:570]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[570:600]).view(-1,1))
## 20
print(torch.FloatTensor(embeddingvalue[600:630]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[630:660]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[660:690]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[690:720]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[720:750]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[750:780]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[780:810]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[810:840]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[840:870]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[870:900]).view(-1,1))
## 30
print(torch.FloatTensor(embeddingvalue[900:930]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[930:960]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[960:990]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[990:1020]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1020:1050]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1050:1080]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1080:1110]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1110:1140]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1140:1170]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1170:1200]).view(-1,1))
## 40
print(torch.FloatTensor(embeddingvalue[1200:1230]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1230:1260]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1260:1290]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1290:1320]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1320:1350]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1350:1380]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1380:1410]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1410:1440]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1440:1470]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1470:1500]).view(-1,1))
## 50
print(torch.FloatTensor(embeddingvalue[1500:1530]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1530:1560]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1560:1590]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1590:1620]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1620:1650]).view(-1,1))
print(torch.FloatTensor(embeddingvalue[1650:1680]).view(-1,1))
## 56

## 2D and 3D embeddingvalue
# print(np.array(embeddingvalue[0:30]).reshape(30,1))
# print(np.array(embeddingvalue[30:60]).reshape(30,1))
# print(np.array(embeddingvalue[60:90]).reshape(30,1))
# print(np.array(embeddingvalue[90:120]).reshape(30,1))
# print(np.array(embeddingvalue[120:150]).reshape(30,1))
# print(np.array(embeddingvalue[150:180]).reshape(30,1))
# print(np.array(embeddingvalue[180:210]).reshape(30,1))
# print(np.array(embeddingvalue[210:240]).reshape(30,1))
# print(np.array(embeddingvalue[240:270]).reshape(30,1))
# print(np.array(embeddingvalue[270:300]).reshape(30,1))
# ## 10
# print(np.array(embeddingvalue[300:330]).reshape(30,1))
# print(np.array(embeddingvalue[330:360]).reshape(30,1))
# print(np.array(embeddingvalue[360:390]).reshape(30,1))
# print(np.array(embeddingvalue[390:420]).reshape(30,1))
# print(np.array(embeddingvalue[420:450]).reshape(30,1))
# print(np.array(embeddingvalue[450:480]).reshape(30,1))
# print(np.array(embeddingvalue[480:510]).reshape(30,1))
# print(np.array(embeddingvalue[510:540]).reshape(30,1))
# print(np.array(embeddingvalue[540:570]).reshape(30,1))
# print(np.array(embeddingvalue[570:600]).reshape(30,1))
# ## 20
# print(np.array(embeddingvalue[600:630]).reshape(30,1))
# print(np.array(embeddingvalue[630:660]).reshape(30,1))
# print(np.array(embeddingvalue[660:690]).reshape(30,1))
# print(np.array(embeddingvalue[690:720]).reshape(30,1))
# print(np.array(embeddingvalue[720:750]).reshape(30,1))
# print(np.array(embeddingvalue[750:780]).reshape(30,1))
# print(np.array(embeddingvalue[780:810]).reshape(30,1))
# print(np.array(embeddingvalue[810:840]).reshape(30,1))
# print(np.array(embeddingvalue[840:870]).reshape(30,1))
# print(np.array(embeddingvalue[870:900]).reshape(30,1))
# ## 30
# print(np.array(embeddingvalue[900:930]).reshape(30,1))
# print(np.array(embeddingvalue[930:960]).reshape(30,1))
# print(np.array(embeddingvalue[960:990]).reshape(30,1))
# print(np.array(embeddingvalue[990:1020]).reshape(30,1))
# print(np.array(embeddingvalue[1020:1050]).reshape(30,1))
# print(np.array(embeddingvalue[1050:1080]).reshape(30,1))
# print(np.array(embeddingvalue[1080:1110]).reshape(30,1))
# print(np.array(embeddingvalue[1110:1140]).reshape(30,1))
# print(np.array(embeddingvalue[1140:1170]).reshape(30,1))
# print(np.array(embeddingvalue[1170:1200]).reshape(30,1))
# ## 40
# print(np.array(embeddingvalue[1200:1230]).reshape(30,1))
# print(np.array(embeddingvalue[1230:1260]).reshape(30,1))
# print(np.array(embeddingvalue[1260:1290]).reshape(30,1))
# print(np.array(embeddingvalue[1290:1320]).reshape(30,1))
# print(np.array(embeddingvalue[1320:1350]).reshape(30,1))
# print(np.array(embeddingvalue[1350:1380]).reshape(30,1))
# print(np.array(embeddingvalue[1380:1410]).reshape(30,1))
# print(np.array(embeddingvalue[1410:1440]).reshape(30,1))
# print(np.array(embeddingvalue[1440:1470]).reshape(30,1))
# print(np.array(embeddingvalue[1470:1500]).reshape(30,1))
# ## 50
# print(np.array(embeddingvalue[1500:1530]).reshape(30,1))
# print(np.array(embeddingvalue[1530:1560]).reshape(30,1))
# print(np.array(embeddingvalue[1560:1590]).reshape(30,1))
# print(np.array(embeddingvalue[1590:1620]).reshape(30,1))
# print(np.array(embeddingvalue[1620:1650]).reshape(30,1))
# print(np.array(embeddingvalue[1650:1680]).reshape(30,1))
# ## 56

tensor([[-1.4705],
        [-1.1177],
        [-1.2675],
        [-1.3337],
        [-1.2333],
        [-1.0485],
        [-1.1953],
        [-1.1079],
        [-1.1256],
        [-1.0750],
        [-1.0798],
        [-1.2163],
        [-1.1848],
        [-1.1422],
        [-1.0354],
        [-1.2469],
        [-0.9826],
        [-0.8923],
        [-0.8298],
        [-1.1400],
        [-1.0137],
        [-0.9953],
        [-1.0840],
        [-1.0784],
        [-1.1303],
        [-0.8711],
        [-0.9670],
        [-1.0763],
        [-1.2059],
        [-1.1065]])
tensor([[-2.8820],
        [-2.5666],
        [-2.6370],
        [-2.6987],
        [-2.6900],
        [-2.5460],
        [-2.7239],
        [-2.8269],
        [-2.5692],
        [-2.5186],
        [-2.2999],
        [-2.4764],
        [-2.7000],
        [-2.5623],
        [-2.2263],
        [-2.5304],
        [-2.1524],
        [-2.2546],
        [-2.2580],
        [-2.3520],
        [-2.3795],
        [-2.3391],
        [-2

        [-4.5235]])
tensor([[-4.9410],
        [-4.5354],
        [-4.6100],
        [-4.7358],
        [-4.6321],
        [-4.5634],
        [-4.7514],
        [-4.9084],
        [-4.5582],
        [-4.7314],
        [-4.4345],
        [-4.5376],
        [-4.8058],
        [-4.6814],
        [-4.7279],
        [-5.0175],
        [-4.7668],
        [-4.5810],
        [-4.3216],
        [-4.9768],
        [-4.7507],
        [-4.3618],
        [-4.5297],
        [-4.8870],
        [-4.7970],
        [-4.4449],
        [-4.9558],
        [-4.8285],
        [-4.7090],
        [-4.5706]])
tensor([[-4.9836],
        [-4.5787],
        [-4.6470],
        [-4.7742],
        [-4.6690],
        [-4.6024],
        [-4.7917],
        [-4.9488],
        [-4.5949],
        [-4.7754],
        [-4.4708],
        [-4.5740],
        [-4.8458],
        [-4.7217],
        [-4.7728],
        [-5.0627],
        [-4.8115],
        [-4.6220],
        [-4.3576],
        [-5.0147],
        [-4.7880],
        [-

        [-4.9999]])
tensor([[-5.3780],
        [-4.9829],
        [-5.0501],
        [-5.1709],
        [-5.0500],
        [-5.0108],
        [-5.2005],
        [-5.3495],
        [-4.9864],
        [-5.2032],
        [-4.8714],
        [-4.9763],
        [-5.2393],
        [-5.1281],
        [-5.2517],
        [-5.5191],
        [-5.2207],
        [-5.0419],
        [-4.7692],
        [-5.4669],
        [-5.2360],
        [-4.8151],
        [-4.9836],
        [-5.3776],
        [-5.2579],
        [-4.8903],
        [-5.5410],
        [-5.2621],
        [-5.2077],
        [-5.0179]])
tensor([[-5.3972],
        [-5.0024],
        [-5.0699],
        [-5.1893],
        [-5.0685],
        [-5.0303],
        [-5.2184],
        [-5.3677],
        [-5.0041],
        [-5.2250],
        [-4.8919],
        [-4.9939],
        [-5.2591],
        [-5.1464],
        [-5.2742],
        [-5.5426],
        [-5.2480],
        [-5.0678],
        [-4.7933],
        [-5.4918],
        [-5.2582],
        [-

        [-5.2011]])
tensor([[-5.5596],
        [-5.1744],
        [-5.2405],
        [-5.3553],
        [-5.2257],
        [-5.1902],
        [-5.3800],
        [-5.5360],
        [-5.1665],
        [-5.4086],
        [-5.0549],
        [-5.1589],
        [-5.4233],
        [-5.3116],
        [-5.4556],
        [-5.7400],
        [-5.4156],
        [-5.2427],
        [-4.9531],
        [-5.6934],
        [-5.4373],
        [-5.0029],
        [-5.1673],
        [-5.6041],
        [-5.4905],
        [-5.0792],
        [-5.7428],
        [-5.4474],
        [-5.4106],
        [-5.2100]])
tensor([[-5.5669],
        [-5.1821],
        [-5.2472],
        [-5.3620],
        [-5.2327],
        [-5.1973],
        [-5.3885],
        [-5.5451],
        [-5.1742],
        [-5.4183],
        [-5.0632],
        [-5.1662],
        [-5.4307],
        [-5.3188],
        [-5.4624],
        [-5.7469],
        [-5.4242],
        [-5.2511],
        [-4.9603],
        [-5.7003],
        [-5.4441],
        [-

In [31]:
#outlet temperature

### testing the integrated model
batch_loss_list = []
tempeture_loss_list = []
RTD_loss_list = []

loss_func = nn.MSELoss()
with torch.no_grad():
    batch_loss = 0
    tempeture_loss = 0
    RTD_loss = 0
    SSRr = 0
    SSRt = 0
    for i in range(len(test_yT)):
        
        targetT = torch.FloatTensor(test_yT[i]).view(-1,1).detach()
        en_input1 = torch.FloatTensor(text_to_onehot(test_one[i])).unsqueeze(0)
        en_input2 = torch.FloatTensor(text_to_onehot(test_two[i])).unsqueeze(0)
        en_input3 = torch.FloatTensor(text_to_onehot(test_three[i])).unsqueeze(0)
        x = torch.cat((en_input1,en_input2,en_input3),0)
        y = torch.FloatTensor(test_quantity[i]).unsqueeze(0)
        d1,d2,d3,outT = my_model(x,y)

        batch_loss += loss_func(outT, targetT).item()
#         RTD_loss += loss_func(outR[0], targetR[0]).item()
        tempeture_loss += loss_func(outT[0], targetT[0]).item()

#         a=torch.ones(1)*16.5239
#         SSRr += loss_func(out[0,0],a).item()
        b=torch.ones(1)*248.679
        SSRt += loss_func(outT[0],b).item()


print(SSRt/(tempeture_loss+SSRt))
print(tempeture_loss/len(test_yT))

0.9652672824210099
0.7901219253829913


In [ ]:
print(len(text_onehot_list))
# print(text_onehot_list.keys())
output = []
for item in text_onehot_list.keys():
#     print(item,':',text_onehot_list[item])
    input_ = torch.FloatTensor(text_onehot_list[item]).unsqueeze(0)
    e1,d1 = my_model.AE(input_)
    print(e1)
    embed = e1.detach().numpy()[0]
    output.append([item, embed])
print(output)